In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


In [17]:
!pip install pyreadline

     |████████████████████████████████| 112kB 1.4MB/s eta 0:00:01
  Running setup.py install for pyreadline ... done


In [3]:
employees = spark.read.csv("HR_employess.csv",header=True)
employees = employees.withColumn("SALARY",employees["SALARY"].cast(DoubleType()))


In [4]:
employees.show(5)

+-----------+----------+---------+--------+------------+-------------------+-------+-------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|          HIRE_DATE| JOB_ID| SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+-------------------+-------+-------+--------------+----------+-------------+
|        100|    Steven|     King|   SKING|515.123.4567|1987-06-17 00:00:00|AD_PRES|24000.0|          null|      null|           90|
|        101|     Neena|  Kochhar|NKOCHHAR|515.123.4568|1989-09-21 00:00:00|  AD_VP|17000.0|          null|       100|           90|
|        102|       Lex|  De Haan| LDEHAAN|515.123.4569|1993-01-13 00:00:00|  AD_VP|17000.0|          null|       100|           90|
|        103| Alexander|   Hunold| AHUNOLD|590.423.4567|1990-01-03 00:00:00|IT_PROG| 9000.0|          null|       102|           60|
|        104|     Bruce|    Ernst|  BERNST|590.423.4568|1991-05-21 00

In [5]:
employees.printSchema()

root
 |-- EMPLOYEE_ID: string (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- EMAIL: string (nullable = true)
 |-- PHONE_NUMBER: string (nullable = true)
 |-- HIRE_DATE: string (nullable = true)
 |-- JOB_ID: string (nullable = true)
 |-- SALARY: double (nullable = true)
 |-- COMMISSION_PCT: string (nullable = true)
 |-- MANAGER_ID: string (nullable = true)
 |-- DEPARTMENT_ID: string (nullable = true)



In [6]:
employees.registerTempTable("employees")

In [7]:
spark.sql("select * from employees").show(2)

+-----------+----------+---------+--------+------------+-------------------+-------+-------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|          HIRE_DATE| JOB_ID| SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+-------------------+-------+-------+--------------+----------+-------------+
|        100|    Steven|     King|   SKING|515.123.4567|1987-06-17 00:00:00|AD_PRES|24000.0|          null|      null|           90|
|        101|     Neena|  Kochhar|NKOCHHAR|515.123.4568|1989-09-21 00:00:00|  AD_VP|17000.0|          null|       100|           90|
+-----------+----------+---------+--------+------------+-------------------+-------+-------+--------------+----------+-------------+
only showing top 2 rows



In [8]:
# 1. Write a query to find the name (first_name, last_name) 
# and the salary of the employees who have a higher salary than the employee whose last_name='Bull'.

spark.sql("select FIRST_NAME,last_name,salary from employees where salary > \
(select salary from employees where last_name='Bull')").show(2)

+----------+---------+-------+
|FIRST_NAME|last_name| salary|
+----------+---------+-------+
|    Steven|     King|24000.0|
|     Neena|  Kochhar|17000.0|
+----------+---------+-------+
only showing top 2 rows



In [9]:
bull_lat_name = employees.filter(col('last_name') == 'Bull').select('salary').rdd.map(lambda x:x.salary).collect()
bull_lat_name

[4100.0]

In [10]:
employees.filter(col('salary')> bull_lat_name[0]).select('FIRST_NAME','last_name','salary').show()

+-----------+---------+-------+
| FIRST_NAME|last_name| salary|
+-----------+---------+-------+
|     Steven|     King|24000.0|
|      Neena|  Kochhar|17000.0|
|        Lex|  De Haan|17000.0|
|  Alexander|   Hunold| 9000.0|
|      Bruce|    Ernst| 6000.0|
|      David|   Austin| 4800.0|
|      Valli|Pataballa| 4800.0|
|      Diana|  Lorentz| 4200.0|
|      Nancy|Greenberg|12000.0|
|     Daniel|   Faviet| 9000.0|
|       John|     Chen| 8200.0|
|     Ismael|  Sciarra| 7700.0|
|Jose Manuel|    Urman| 7800.0|
|       Luis|     Popp| 6900.0|
|        Den| Raphaely|11000.0|
|    Matthew|    Weiss| 8000.0|
|       Adam|    Fripp| 8200.0|
|      Payam| Kaufling| 7900.0|
|     Shanta|  Vollman| 6500.0|
|      Kevin|  Mourgos| 5800.0|
+-----------+---------+-------+
only showing top 20 rows



In [16]:
# 2. Write a query to find the name (first_name, last_name) 
# of all employees who works in the IT department.

spark.sql("select first_name ,last_name from employees 
          where DEPARTMENT_ID in (select DEPARTMENT_ID from DEPARTMENT where DEPARTMENT_NAME = 'IT' ").show()

ParseException: "\nmismatched input 'from' expecting <EOF>(line 1, pos 29)\n\n== SQL ==\nselect first_name ,last_name from employees where DEPARTMENT_ID in (select DEPARTMENT_ID from DEPARTMENT where DEPARTMENT_NAME = 'IT' \n-----------------------------^^^\n"